In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_3.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,"Throughout the world, there are multiple young...",0.0
1,Online classes are not as reliable as classes ...,0.0
2,Imagine going to school to find yourself strug...,0.0
3,Yes i think students would benefit from being ...,0.0
4,Take learning distance on home is not good ide...,0.0
...,...,...
17983,"\n\nChange the Electoral College, These citize...",0.0
17984,"Facial Potion Coding System, a program that UA...",0.0
17985,"Dear Senator,\n\nthe Electoral College was cre...",0.0
17986,The road to the truly Autonomous cur\n\nWouldn...,0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 4096])
Attention Mask size: torch.Size([17988, 4096])
Vocabulary Size: 29866
Max Sequence Length: 4096


In [5]:
model=Sequential()
input_length=4096
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=128,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)

Epoch 1/5
450/450 [==============================] - 1014s 2s/step - loss: 0.0146 - accuracy: 0.9978
Epoch 2/5
450/450 [==============================] - 984s 2s/step - loss: 5.4750e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 840s 2s/step - loss: 2.7638e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 842s 2s/step - loss: 1.6883e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 837s 2s/step - loss: 1.1303e-05 - accuracy: 1.0000


In [6]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 41s 359ms/step - loss: 9.3823e-06 - accuracy: 1.0000


[9.382273674418684e-06, 1.0]

In [7]:
model.save("newmodel3.keras")